In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
! pip install pytorch_lightning --quiet
! pip install neptune-client==0.9.1 --quiet

In [ ]:
!pip install transformers --quiet

In [ ]:
import neptune.new as neptune
run = neptune.init(project='eduardocoraca/aula11', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmMzZmYmY3MC01NWJjLTQ1MjItYmVjMC0zNGNjNGZiMDA4YzcifQ==')

https://app.neptune.ai/eduardocoraca/aula11/e/AL11-2


# Inicialização

## Carregando o Dataset

In [ ]:
!wget -nc http://files.fast.ai/data/examples/imdb_sample.tgz
!tar -xzf imdb_sample.tgz

File ‘imdb_sample.tgz’ already there; not retrieving.



In [ ]:
import pandas as pd
df = pd.read_csv('imdb_sample/texts.csv')
df.shape
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [ ]:
treino = df[df['is_valid'] == False] # este é o df usado para treinamento
test = df[df['is_valid'] == True]

print('treino.shape:', treino.shape)
print('test.shape:', test.shape)

# não usei estes arrays abaixo
X_treino_val = treino['text']
Y_treino_val = treino['label']
X_test = test['text']
Y_test = test['label']

train_split = 0.8

idx = np.arange(0,len(X_treino_val))
np.random.shuffle(idx)

train_idx = idx[0:int(train_split*len(idx))]
valid_idx = idx[int(train_split*len(idx)):]

X_treino = X_treino_val[train_idx]
X_valid = X_treino_val[valid_idx]

Y_treino = Y_treino_val[train_idx]
Y_valid = Y_treino_val[valid_idx]

print(X_treino.shape, Y_treino.shape)
print(X_valid.shape, Y_valid.shape)

mapeamento = {'positive': True, 'negative': False}
Y_treino_bool = Y_treino.map(mapeamento)
Y_valid_bool = Y_valid.map(mapeamento)
Y_test_bool = Y_test.map(mapeamento)
print(Y_treino_bool.head())

treino.shape: (800, 3)
test.shape: (200, 3)
(640,) (640,)
(160,) (160,)
319    False
701    False
298     True
138     True
153     True
Name: label, dtype: bool


In [ ]:
'''X_treino = np.array(X_treino)
X_valid = np.array(X_valid)
X_test = np.array(X_test)

Y_treino = np.array(Y_treino)
Y_valid = np.array(Y_valid)
Y_test = np.array(Y_test) '''

'X_treino = np.array(X_treino)\nX_valid = np.array(X_valid)\nX_test = np.array(X_test)\n\nY_treino = np.array(Y_treino)\nY_valid = np.array(Y_valid)\nY_test = np.array(Y_test) '

## Tokenizer

Utilizando o tokenizador AutoTokenizer

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

Checando o output

In [ ]:
teste = tokenizer(X_treino[0], padding='max_length', truncation=True)
teste.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
len(teste['input_ids'])

512

## Dataset

In [ ]:
!pip install datasets --quiet

In [ ]:
from datasets import Dataset

In [ ]:
treino_df = pd.DataFrame()
treino_df['text'] = treino['text']
treino_df['labels'] = list(map(lambda x: 1 if x=='positive' else 0, treino['label']))

test_df = pd.DataFrame()
test_df['text'] = test['text']
test_df['labels'] = list(map(lambda x: 1 if x=='positive' else 0, test['label']))

In [ ]:
treino_dataset = Dataset.from_pandas(treino_df)
treino_dataset = treino_dataset.remove_columns('__index_level_0__') # removendo coluna criada automaticamente

test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.remove_columns('__index_level_0__') # removendo coluna criada automaticamente

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

treino_dataset_tokenized = treino_dataset.map(tokenize_function, batched=True)
test_dataset_tokenized = test_dataset.map(tokenize_function, batched=True)

treino_dataset_tokenized = treino_dataset_tokenized.shuffle()
test_dataset_tokenized = test_dataset_tokenized.shuffle()

# coloca no formato do PyTorch
#treino_dataset_tokenized.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
#test_dataset_tokenized.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

Conteúdo do Dataset

In [ ]:
treino_dataset_tokenized

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'text', 'token_type_ids'],
    num_rows: 800
})

In [ ]:
len(treino_dataset_tokenized[0]['attention_mask'])

512

In [ ]:
treino_dataset_tokenized[0]['text']

'Warning Spoiler. . . I have to agree with you, it was almost there. This was such a bad movie, about such and interesting true story. It had such promise, but the acting was ridiculous at best. Some sets were beautiful and realistic. Others are something out of a theme park. I found myself laughing as I watched, what was suppose to be, serious scenes. I really wanted to like this movie, but I couldn\'t. The best part was the fight between friends that ended with the "King" dying. I liked the Queens\' punishment. And, the final shot made a beautiful picture, though. There are so many better movies to watch. I don\'t recommend this.'

In [ ]:
treino_dataset_tokenized[0]['input_ids']

[101,
 19680,
 156,
 5674,
 25614,
 119,
 119,
 119,
 146,
 1138,
 1106,
 5340,
 1114,
 1128,
 117,
 1122,
 1108,
 1593,
 1175,
 119,
 1188,
 1108,
 1216,
 170,
 2213,
 2523,
 117,
 1164,
 1216,
 1105,
 5426,
 2276,
 1642,
 119,
 1135,
 1125,
 1216,
 4437,
 117,
 1133,
 1103,
 3176,
 1108,
 9944,
 1120,
 1436,
 119,
 1789,
 3741,
 1127,
 2712,
 1105,
 13142,
 119,
 8452,
 1132,
 1380,
 1149,
 1104,
 170,
 3815,
 2493,
 119,
 146,
 1276,
 1991,
 5574,
 1112,
 146,
 2542,
 117,
 1184,
 1108,
 6699,
 1106,
 1129,
 117,
 3021,
 4429,
 119,
 146,
 1541,
 1458,
 1106,
 1176,
 1142,
 2523,
 117,
 1133,
 146,
 1577,
 112,
 189,
 119,
 1109,
 1436,
 1226,
 1108,
 1103,
 2147,
 1206,
 2053,
 1115,
 2207,
 1114,
 1103,
 107,
 1624,
 107,
 5694,
 119,
 146,
 3851,
 1103,
 9347,
 112,
 7703,
 119,
 1262,
 117,
 1103,
 1509,
 2046,
 1189,
 170,
 2712,
 3439,
 117,
 1463,
 119,
 1247,
 1132,
 1177,
 1242,
 1618,
 5558,
 1106,
 2824,
 119,
 146,
 1274,
 112,
 189,
 18029,
 1142,
 119,
 102,
 0,
 0,
 0

Modelo

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

# Treinamento

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer",
                                  evaluation_strategy = "epoch",
                                  gradient_accumulation_steps = 2,
                                  num_train_epochs = 10,
                                  learning_rate = 1e-5,
                                  logging_strategy = "epoch")

Função para computar a métrica no final da época

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    met = metric.compute(predictions=predictions, references=labels)
    run['train/accuracy'].log(met['accuracy'])
    return met

In [ ]:
from transformers import Trainer

trainer = Trainer(model = model,
                  args = training_args,
                  train_dataset = treino_dataset_tokenized,
                  eval_dataset = test_dataset_tokenized,
                  compute_metrics = compute_metrics)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.679100,0.643178,0.660000
2,0.502300,0.395669,0.850000
3,0.278100,0.295412,0.905000
4,0.160700,0.309373,0.900000
5,0.102100,0.360325,0.905000
6,0.077500,0.380859,0.890000
7,0.051100,0.427795,0.890000
8,0.036300,0.467688,0.890000
9,0.031800,0.482057,0.890000
10,0.030300,0.486034,0.885000


TrainOutput(global_step=500, training_loss=0.19491549730300903, metrics={'train_runtime': 918.09, 'train_samples_per_second': 0.545, 'total_flos': 0, 'epoch': 10.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 434824704, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 28672, 'train_mem_gpu_alloc_delta': 1364854272, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 6506152960})